In [ ]:
#Import libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings
import random
warnings.filterwarnings('ignore')
import sklearn
import sys
sys.path
sys.path.append('/Users/matthewmcteer/opt/anaconda3/lib/python3.7/site-packages')
import shap
import xgboost as xgb

from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import time
import missingno as msno



# Response 11 - MASL vs MASH

In [ ]:
#Upload Dataset
#Only considering Baseline Event Types
df = pd.read_csv(#Data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_11'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where MASL vs MASH = NaN
df = df[features]
df = df[df['response_11'].notna()]

df['response_11'].value_counts()


In [ ]:
df.shape

In [ ]:
#Bar chart displaying level of completeness for each feature
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [ ]:
#Save Baseline, Metacohort Dataset
df.to_csv(#Saved data)

## XGBoost

In [ ]:
df = pd.read_csv(#Upload saved data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_11'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_11'].notna()]

df['response_11'].value_counts()



In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'MASL vs. MASH']



In [ ]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:19]
y = df['MASL vs. NASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))



In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - MASL vs MASH")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#Upload imputed dataset)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'MASL vs. MASH']
df.shape


In [ ]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['MASL vs. MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],
)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - MASL vs. NASH")
plt.show()




## XGBoost + MICE + SMOTE

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('Core_Response11_METACOHORT_MICE.csv')
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'MASL vs MASH]
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['MASL vs. NASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

unique, counts = np.unique(y_train_res, return_counts=True)
print(np.asarray((unique, counts)).T)
print(y_test.value_counts())


In [ ]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(model, X_train_res, y_train_res, cv=kfold)
tn, fp, fn, tp = confusion_matrix(y_train_res, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)
print('sensitivity:', sens)
print('specificity:', spec)

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - MASL vs. MASH")
plt.show()



# Response 12 - At-Risk MASH

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#Read in dataset)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_12'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_12 = NaN
df = df[features]
df = df[df['response_12'].notna()]

df['response_12'].value_counts()

#Save data
df.to_csv(#Save data)



## XGBoost

In [ ]:
df = pd.read_csv(#data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_7'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_12 = NaN
df = df[features]
df = df[df['response_12'].notna()]

df['response_12'].value_counts()

df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'At-Risk MASH']





In [ ]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['At-Risk MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - At-Risk MASH")
plt.show()



## XGBoost + MICE 

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'At-Risk MASH']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['At-Risk MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - At-Risk MASH")
plt.show()




## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Historic Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'At-Risk MASH']
df.shape

In [ ]:
#Create train and test sets
X = df.iloc[:,:19]
y = df['At-Risk MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)
print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - At-Risk MASH")
plt.show()



# Response 13 - High Activity

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read in data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_13'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_13'].notna()]

df['response_13'].value_counts()

df.to_csv(#save data)





## XGBoost

In [ ]:
df = pd.read_csv(#read data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_13'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_13 = NaN
df = df[features]
df = df[df['response_13'].notna()]

df['response_13'].value_counts()

df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'High Activity']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['High Activity']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - High Activity")
plt.show()






## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'High Activity']
df.shape

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['High Activity']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - High Activity")
plt.show()




## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'High Activity']
df.shape

In [ ]:
#Create train and test sets
X = df.iloc[:,:19]
y = df['High Activity']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - High Activity")
plt.show()





# Response 14 - Clinically Significant Fibrosis

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']



In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_14'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_14'].notna()]

df['response_14'].value_counts()

df.to_csv(#save data)




## XGBoost 

In [ ]:
df = pd.read_csv('Core_Response14_METACOHORT.csv')

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_14'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_14 = NaN
df = df[features]
df = df[df['response_14'].notna()]

df['response_14'].value_counts()

df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Clinically Significant Fibrosis']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Clinically Significant Fibrosis']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - Clinically Significant Fibrosis")
plt.show()





## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed dataset)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Clinically Significant Fibrosis']
df.shape

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Clinically Significant Fibrosis']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - Clinically Significant Fibrosis")
plt.show()




## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed dataset)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Clinically Significant Fibrosis']
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['Clinically Significant Fibrosis']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)
print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - Clinically Significant Fibrosis")
plt.show()




# Response 15  - Advanced Fibrosis (Histology Confirmed)

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read dataset)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_15'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_15'].notna()]

df['response_15'].value_counts()

df.to_csv(#save data)





## XGBoost

In [ ]:
df = pd.read_csv(#read data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_15'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_15'].notna()]

df['response_15'].value_counts()

df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Advanced Fibrosis (Histology Confirmed)']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Advanced Fibrosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - Advanced Fibrosis (Histology Confirmed)")
plt.show()




## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Advanced Fibrosis (Histology Confirmed)']
df.shape

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Advanced Fibrosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - Advanced Fibrosis (Histology Confirmed)")
plt.show()








## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Advanced Fibrosis (Histology Confirmed)']
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['Advanced Fibrosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)
print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - Advanced Fibrosis (Histology Confirmed)")
plt.show()



# Response 16 - Cirrhosis (Histology Confirmed)

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_16'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_16'].notna()]

df['response_16'].value_counts()
df.to_csv(#save data)






## XGBoost 

In [ ]:
df = pd.read_csv(#read data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_16'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_16 = NaN
df = df[features]
df = df[df['response_16'].notna()]

df['response_16'].value_counts()
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Cirrhosis (Histology Confirmed)']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Cirrhosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - Cirrhosis (Histology Confirmed)")
plt.show()









## XGBoost + MICE

In [ ]:
df = pd.read_csv(# read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Cirrhosis (Histology Confirmed)']
df.shape


In [ ]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Cirrhosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - Cirrhosis (Histology Confirmed)")
plt.show()




## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Cirrhosis (Histology Confirmed)']
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['Cirrhosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)
print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - Cirrhosis (Histology Confirmed)")
plt.show()




# Response 17 - Advanced Fibrosis (Histology & Clinically Confirmed)

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_17'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_17'].notna()]

df['response_17'].value_counts()

df.to_csv(#save data)



## XGBoost

In [ ]:
df = pd.read_csv('Core_Response17_METACOHORT.csv')

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_17'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_17 = NaN
df = df[features]
df = df[df['response_17'].notna()]

df['response_17'].value_counts()

df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Advanced Fibrosis (Histology & Clinical)']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Advanced Fibrosis (Histology & Clinical)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - Advanced Fibrosis (Histology & Clinically Confirmed)")
plt.show()







## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Advanced Fibrosis (Histology & Clinical)']
df.shape

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Advanced Fibrosis (Histology & Clinical)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - Advanced Fibrosis (Histology & Clinically Confirmed)")
plt.show()





## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(# read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Advanced Fibrosis (Histology & Clinical)']
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['Advanced Fibrosis (Histology & Clinical)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - Advanced Fibrosis (Histology & Clinically Confirmed)")
plt.show()







# Response 18 - Cirrhosis (Histology & Clinically Confirmed)

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_18'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_18 = NaN
df = df[features]
df = df[df['response_18'].notna()]

df['response_18'].value_counts()
df.to_csv(#save data)




## XGBoost

In [ ]:
df = pd.read_csv('Core_Response18_METACOHORT.csv')

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_18'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_18 = NaN
df = df[features]
df = df[df['response_18'].notna()]

df['response_18'].value_counts()


df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Cirrhosis (Histology & Clinically Confirmed)']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Cirrhosis (Histology & Clinically Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=None, colsample_bytree=0.75, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0.0, max_depth=5,
              min_child_weight=1.0, monotone_constraints=None,
              n_estimators=250, n_jobs=-1, num_parallel_tree=None,
              random_state=42, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=1.0, subsample=None, tree_method='auto',
              validate_parameters=False, verbosity=None)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - Cirrhosis (Histology & Clinically Confirmed)")
plt.show()




## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed dataset)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Cirrhosis (Histology & Clinically Confirmed)']
df.shape

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['Cirrhosis (Histology & Clinically Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - Cirrhosis (Histology & Clinically Confirmed)")
plt.show()





## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'Cirrhosis (Histology & Clinically Confirmed)']
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['Cirrhosis (Histology & Clinically Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - Cirrhosis (Histology & Clinically Confirmed)")
plt.show()




# Response 20 - At-Risk MASLD

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_20'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_20'].notna()]

df['response_20'].value_counts()

df.to_csv(#save data)



## XGBoost

In [ ]:
df = pd.read_csv(#read data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
            'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'idf_metabolic_syndrome',
            'eGFR',                     
            'dyslipidaemia',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',      
            'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_CD_OSA',
            'AST_ALT_Ratio',                    
                             ],
    
    'target':[
        'response_20'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_20'].notna()]

df['response_20'].value_counts()

df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'response_20']

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Create train and test sets  
X = df.iloc[:,:19]
y = df['response_20']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Core Variables - At-Risk MASLD")
plt.show()







## XGBoost + MICE

In [ ]:
df = pd.read_csv(# read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'response_20']
df.shape

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)
#Create train and test sets  
X = df.iloc[:,:19]
y = df['response_20']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Core Variables - At-Risk MASLD")
plt.show()





## XGBoost + MICE + SMOTE

In [ ]:
df = pd.read_csv(#read imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'ALT', 'AST', 'GGT', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'Obstructive Sleep Apnoea', 'AST-ALT Ratio',
              'response_20']
df.shape

#Create train and test sets
X = df.iloc[:,:19]
y = df['response_20']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res , plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Core Variables - At-Risk MASLD")
plt.show()





